In [ ]:
from flask import Flask,request,jsonify
import numpy as np
from tensorflow import keras
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp
import numpy as np 
import os
import pickle
from PIL import Image
import base64
from io import BytesIO
from flask import Flask, render_template, request
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from PIL import Image
import numpy as np
#Arabic
model_fileTrain = "C:/Users/DELL/Downloads/model/ARSLTrainModel.tflite"
model_fileCafe = "C:/Users/DELL/Downloads/model/ARSLCafeModel.tflite"
model_fileBank = "C:/Users/DELL/Downloads/model/ARSLBankModel.tflite"
model_fileHospital = "C:/Users/DELL/Downloads/model/ARSLHospitalModel.tflite"

model_fileGreek = "C:/Users/DELL/Downloads/model/GreekModel.tflite"
model_fileTur = "C:/Users/DELL/Downloads/model/TurkishModel.tflite"
model_fileIndian = "C:/Users/DELL/Downloads/model/IndianVids.tflite"
model_file2 = "C:/Users/DELL/Downloads/model/modelar.tflite"
model_file3 = "C:/Users/DELL/Downloads/model/modelen.tflite"
modelTur = "C:/Users/DELL/Downloads/model/model_tur.tflite"

#indian 
interpreterInd = tf.lite.Interpreter(model_path=model_fileIndian)
interpreterInd.allocate_tensors()
input_detailsInd = interpreterInd.get_input_details()
output_detailsInd = interpreterInd.get_output_details()

#ArabBank
interpreterBank = tf.lite.Interpreter(model_path=model_fileBank)
interpreterBank.allocate_tensors()
input_detailsBank = interpreterBank.get_input_details()
output_detailsBank = interpreterBank.get_output_details()
#ArabCafe
interpreterCafe = tf.lite.Interpreter(model_path=model_fileCafe)
interpreterCafe.allocate_tensors()
input_detailsCafe = interpreterCafe.get_input_details()
output_detailsCafe = interpreterCafe.get_output_details()
#ArabTrain
interpreterTrain = tf.lite.Interpreter(model_path=model_fileTrain)
interpreterTrain.allocate_tensors()
input_detailsTrain = interpreterTrain.get_input_details()
output_detailsTrain = interpreterTrain.get_output_details()
#ArabHosp
interpreterHospital = tf.lite.Interpreter(model_path=model_fileHospital)
interpreterHospital.allocate_tensors()
input_detailsHospital = interpreterHospital.get_input_details()
output_detailsHospital = interpreterHospital.get_output_details()
app = Flask(__name__)
UPLOAD_FOLDER = 'videos'
# Create the folder if it does not exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# Set the upload folder as an app configuration
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
@app.route('/')
def index():
    return "Welcome To Our Server"
@app.route('/predictBank',methods=['POST'])
def predictBank():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    #///////////////////////////////////////////////////////////////////////////////////////
    interpreterBank.set_tensor(input_detailsBank[0]['index'], X.astype(np.float32))
    interpreterBank.invoke()
    output_data = interpreterBank.get_tensor(output_detailsBank[0]['index'])
    result = np.argmax(output_data[0])
    word = np.array(['on', 'Bank manager', 'Contract', 'Visa', 'Driving License', 'account', 'receive', 'in', 'money', 'my', 'Thank god', 'birth certificate', 'Withdraw money', 'doing', 'finance', 'Bank', 'loan', 'Form', 'Cairo', 'Shop License', 'deaf', 'deposit', 'rent', 'your', 'and', 'private', 'ownership', 'what', 'monthly payment', 'conditions', 'adress', 'project', 'authorization', 'ok', 'hello', 'help', 'Integrated services card', 'or', 'branch', 'question', 'Revenues', 'housing', 'salary', 'fine', 'Financial benefits', 'changing', 'open', 'Your health is fineΓò¬╞Æ', 'How much', 'because', 'Id card', 'problem', 'wait', 'expenses', 'need', 'pension', 'me', 'you', 'yes', 'rejection', 'savings book', 'Alexandria', 'apartment', 'How are you', 'money transfer', 'name', 'support', 'window', 'qualification', 'general', 'new', 'governorate', 'Death certificate', 'Sign language interpreter', 'signature', 'number'])
    return jsonify({'placement':str(word[result])})


@app.route('/predictCafe',methods=['POST'])
def predictCafe():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    #///////////////////////////////////////////////////////////////////////////////////////
    interpreterCafe.set_tensor(input_detailsCafe[0]['index'], X.astype(np.float32))
    interpreterCafe.invoke()
    output_data = interpreterCafe.get_tensor(output_detailsCafe[0]['index'])
    result = np.argmax(output_data[0])
    word = np.array(['fine', 'glass', 'ok', 'me', 'in', 'milk', 'water', 'banana', 'cold', 'drink', 'How much', 'question', 'adress', 'no', 'anise', 'cinnamon', 'name', 'berell', 'hello', 'you', 'number', 'hot', 'Thank god', 'doing', 'How are you', 'juice', 'wait', 'nescafe', 'lemon', 'need', 'on', 'problem', 'pepsi', 'tea', 'spoon', 'deaf', 'mint', 'mango', 'help', 'or', 'orange', 'coffee', 'your', 'and', 'Your health is fine╪ƒ', 'Roselle', 'what', 'new', 'my', 'money', 'strawberry', 'yes', 'birth certificate', 'fenugreek'])
    return jsonify({'placement':str(word[result])})



@app.route('/predictTrain',methods=['POST'])
def predictTrain():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    #///////////////////////////////////////////////////////////////////////////////////////
    interpreterTrain.set_tensor(input_detailsTrain[0]['index'], X.astype(np.float32))
    interpreterTrain.invoke()
    output_data = interpreterTrain.get_tensor(output_detailsTrain[0]['index'])
    result = np.argmax(output_data[0])
    word = np.array(['Banha', 'Train station', 'Ismailia', 'Id card', 'question', 'Damanhur', 'Thank god', 'Tanta', 'Kafr El-Sheikh', 'discount', 'Mansoura', 'How much', 'Sign language interpreter', 'problem', 'on', 'Qena', 'or', 'Sohag', 'Aswan', 'ok', 'Giza', 'Alexandria', 'because', 'Port Said', 'Marsa Matrouh', 'where', 'me', 'delay', 'help', 'what', 'fast', 'money', 'I go', 'new', 'Bani Sweif', 'in', 'my', 'yes', 'doing', 'hour', 'hello', 'deaf', 'number', 'and', 'wait', 'after', 'Integrated services card', 'How are you', 'Damietta', 'before', 'luxor', 'place', 'late', 'Cairo', 'ticket', 'adress', 'Train', 'He will come', 'birth certificate', 'together', 'need', 'name', 'Form', 'Fayoum', 'Suez', 'you', 'fine', 'Minya', 'when', 'Asyut', 'your'])
    return jsonify({'placement':str(word[result])})



@app.route('/predictHospital',methods=['POST'])
def predictHospital():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    #///////////////////////////////////////////////////////////////////////////////////////
    interpreterHospital.set_tensor(input_detailsHospital[0]['index'], X.astype(np.float32))
    interpreterHospital.invoke()
    output_data = interpreterHospital.get_tensor(output_detailsHospital[0]['index'])
    result = np.argmax(output_data[0])
    word = np.array(['Obstetrics and gynecology', 'and', 'what', 'overheat', 'on', 'hospital', 'feed', 'ointment', 'Form', 'rays', 'Congestion', 'burn', 'eyes', 'room', 'How are you', 'gauze', 'illness', 'vacation', 'my', 'nusre', 'adress', 'syrup', 'service', 'new', 'heart', 'Sign language interpreter', 'Diabetes', 'birth', 'physical therapy', 'deaf', 'Tell me', 'doctor', 'Anemia', 'sick', 'brain', 'blood', 'because', 'Health Care', 'Forbidden', 'pee', 'tired', 'Thank god', 'headache', 'your', 'Diarrhea', 'ok', 'ticket', 'colic', 'skin', 'Ear, Nose and Throat', 'fracture', 'or', 'poop', 'wait', 'doing', 'question', 'number', 'in', 'you', 'name', 'corona', 'Emergency', 'Injection', 'health insurance', 'scissors', 'nerves', 'problem', 'fine', 'birth certificate', 'tab', 'analysis', 'vertigo', 'bones', 'money', 'Recepion', 'Pregnancy', 'How much', 'liver', 'appendicitis', 'Id card', 'mask', 'hello', 'cold', 'canula', 'cotton', 'model', 'pressure', 'children', 'surgery', 'Your health is fineΓò¬╞Æ', 'paper', 'Poisoning', 'section', 'virus', 'me', 'constipation', 'esoteric', 'Heart attack', 'Integrated services card', 'yes', 'help', 'need'])
    return jsonify({'placement':str(word[result])})



#Greek
interpreterGreek = tf.lite.Interpreter(model_path=model_fileGreek)
interpreterGreek.allocate_tensors()
input_detailsGreek = interpreterGreek.get_input_details()
output_detailsGreek = interpreterGreek.get_output_details()

@app.route('/predictGreek',methods=['POST'])
def predictGreek():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    interpreterGreek.set_tensor(input_detailsGreek[0]['index'], X.astype(np.float32))
    interpreterGreek.invoke()
    output_data = interpreterGreek.get_tensor(output_detailsGreek[0]['index'])
    result = np.argmax(output_data[0])
    return jsonify({'placement':str(getcodeGreek(result))})


#tur

@app.route('/predictTur',methods=['POST'])
def predictTur():
    interpreterTur = tf.lite.Interpreter(model_path=model_fileTur)
    interpreterTur.allocate_tensors()
    input_detailsTur = interpreterTur.get_input_details()
    output_detailsTur = interpreterTur.get_output_details()
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
    mp_holistic = mp.solutions.holistic
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    interpreterTur.set_tensor(input_detailsTur[0]['index'], X.astype(np.float32))
    interpreterTur.invoke()
    output_data = interpreterTur.get_tensor(output_detailsTur[0]['index'])
    result = np.argmax(output_data[0])
    return jsonify({'placement':str(getcodeTur(result))})
#______________________________________________________________________
@app.route('/predictIndian',methods=['POST'])
def predictIndian():
    
#     frames = request.body('Frames')
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    for frame in frames:
        image_value={}
        for key, value in frame.items():
            image_value = value
        image_data = base64.b64decode(image_value['Image'])
        img = Image.open(BytesIO(image_data))
        image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
        counter+=1
        img.save(image_path, format="JPEG")
        framelist.append(f'{image_path}')
#     counter = 0
#     pathToFrames = []
#     for frame in frames:
#         filepath = os.path.join(app.config['UPLOAD_FOLDER'], f'{counter}.bmp')
#         frame.save(filepath)
#         pathToFrames.append(filepath)
#         counter++
#     framList = sorted(os.listdir(os.path.join(app.config['UPLOAD_FOLDER'])))
    mp_holistic = mp.solutions.holistic
    #_________________________________________________
#     framList = []
#     video_capture = cv2.VideoCapture(vid_file)
#     ret, frame = video_capture.read()
#     while ret:
#         if frame is not None:
#             frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
#             framList.append(frame)
#         ret, frame = video_capture.read()
#     video_capture.release()
    #________________________________________________
    #*******************************************************************
    pose_keypoints, lh_keypoints, rh_keypoints = [], [], []
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      # Loop through the video frames
      for frame in framelist:
    #         image, results = mediapipe_detection(frame, holistic)
            #///////////////////////////////////////////////////////
            image = cv2.imread(frame)
            if image is None:
                raise ValueError(f"Failed to load image: {image}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
            #///////////////////////////////////////////////////////

            # Extract keypoints
    #         pose, lh, rh = extract_keypoints(results)
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            nose=pose[:3]
            lh_wrist=lh[:3]
            rh_wrist=rh[:3]
    #         pose = adjust_landmarks(pose,nose)
            ###########################################################
            arr_reshaped = pose.reshape(-1, 3)
            center_repeated = np.tile(nose, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            pose = arr_adjusted.reshape(-1)
            ###########################################################
    #         lh = adjust_landmarks(lh,lh_wrist)
            ###########################################################
            arr_reshaped = lh.reshape(-1, 3)
            center_repeated = np.tile(lh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            lh = arr_adjusted.reshape(-1)
            ###########################################################
    #         rh = adjust_landmarks(rh,rh_wrist)
            ###########################################################
            arr_reshaped = rh.reshape(-1, 3)
            center_repeated = np.tile(rh_wrist, (len(arr_reshaped), 1))
            arr_adjusted = arr_reshaped - center_repeated
            rh = arr_adjusted.reshape(-1)
            ###########################################################
            #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
            # Add the keypoints to the list for this video
            pose_keypoints.append(pose)
            lh_keypoints.append(lh)
            rh_keypoints.append(rh)     
    #*********************************************************************
    # Initialize the lists of sequences and labels
    sequences = []
    labels = []

    # Iterate over the words

    # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
    # Load the left hand array
    res_lh = np.array(lh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_lh.shape[0], 48)
    res_lh = res_lh[:num_frames, :]
    while num_frames < 48:
        res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the right hand array
    res_rh =  np.array(rh_keypoints)

    # Determine how many frames to select
    num_frames = min(res_rh.shape[0], 48)
    res_rh = res_rh[:num_frames, :]
    while num_frames < 48:
        res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Load the pose array
    res_pose =np.array(pose_keypoints) 

    # Determine how many frames to select
    num_frames = min(res_pose.shape[0], 48)
    res_pose = res_pose[:num_frames, :]
    while num_frames < 48:
        res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
        num_frames += 1

    # Append the subsequence to the list of sequences
    sequences.append(np.concatenate((res_pose, res_lh, res_rh), axis=1))
    # Append the label to the list of labels

    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    interpreterInd.set_tensor(input_detailsInd[0]['index'], X.astype(np.float32))
    interpreterInd.invoke()
    output_data = interpreterInd.get_tensor(output_detailsInd[0]['index'])
    result = np.argmax(output_data[0])
    return jsonify({'placement':str(getcodeIndian(result))})

def getcodeIndian(n):
    code={'Bird': 0, 'Energy': 1, 'Price': 2, 'Race': 3, 'Sign': 4, 'Cat': 5, 'Newspaper': 6, 'War': 7, 'Bill': 8, 'Technology': 9, 'God': 10, 'Fish': 11, 'Peace': 12, 'Horse': 13, 'Team': 14, 'Exercise': 15, 'Death': 16, 'Money': 17, 'Marriage': 18, 'Medicine': 19, 'Gun': 20, 'Cow': 21, 'Mouse': 22, 'Ball': 23, 'Religion': 24, 'Attack': 25, 'Animal': 26, 'Dog': 27, 'Election': 28, 'Science': 29, 'Sport': 30}

    for x,y in code.items():
        if n==y:
            return x


def getcodeGreek(n):
    code={'note': 0, 'smile': 1, 'rucksack': 2, 'sky': 3, 'send': 4, 'release': 5, 'shelf': 6, 'round': 7, 'remove': 8, 'right': 84, 'fly': 10, 'relative': 11, 'recipe': 12, 'rich': 13, 'quite': 14, 'let': 15, 'RAW': 16, 'sick': 17, 'air': 18, 'rare': 19, 'saturday': 20, 'schedule': 21, '17': 62, 'smell': 23, 'season': 24, 'remember': 25, 'ship': 26, 'return': 27, 'brief': 28, 'say': 29, 'marriage': 31, 'never': 32, 'shout': 33, 'remind': 34, 'small': 35, 'shirt': 36, 'roof': 37, 'sheet': 38, 'on top of': 39, 'seat': 40, 'sell': 41, 'room': 42, 'scientist': 43, 'salt': 44, 'ladder': 45, 'sleep': 46, 'search': 47, 'group': 48, 'share': 51, 'refrigerator': 52, 'sharp': 53, 'short': 54, 'real': 55, 'signal': 56, 'ribbon': 57, 'radiator': 58, 'result': 59, 'from time to time': 60, 'slowly': 61, 'size': 63, 'same': 64, 'save': 65, 'ruin': 66, 'radio': 67, 'sea': 68, 'rain': 69, 'shoe': 70, 'show': 71, 'ring': 72, 'resign': 73, 'rice': 74, 'repeat': 75, 'bag': 76, 'reason': 77, 'sit': 78, 'see': 79, 'serve': 80, 'relation': 81, 'scratch': 82, 'skirt': 83, 'simple': 85, 'responsible': 86, 'river': 87, 'birthday': 88, 'skin': 89, 'drop': 90, 'root': 91, 'similar': 92, 'shop': 93, 'read': 94, 'salad': 95, 'almost': 96, 'reach': 97, 'language': 98, 'shine': 99, 'robust': 100}
    for x,y in code.items():
        if n==y:
            return x
def getcodeTur(n):
    code={'male': 0, 'enjoy_your_meal': 1, 'gift': 2, 'glass': 3, 'pharmacy': 4, 'change': 5, 'heavy': 6, 'drink': 7, 'tree': 8, 'elephant': 9, 'labor': 10, 'wait': 11, 'government': 12, 'cry': 13, 'meal': 14, 'push': 15, 'key': 16, 'wallet': 17, 'minute': 18, 'not_interested': 19, 'single': 20, 'memorize': 21, 'always': 22, 'saturday': 23, 'tea': 24, 'child': 25, 'together': 26, 'ataturk': 27, 'lightweight': 28, 'honey': 29, 'yesterday': 30, 'yes': 31, 'hammer': 32, 'medicine': 33, 'flag': 34, 'hurry': 35, 'same': 36, 'retired': 37, 'we': 38, 'mirror': 39, 'hospital': 40, 'baby': 41, 'wise': 42, 'animal': 43, 'topple': 44, 'ill': 45, 'laugh': 46, 'shirt': 47, 'football': 48, 'fault': 49, 'ugly': 50, 'unwise': 51, 'wednesday': 52, 'fork': 53, 'soup': 54, 'shoe': 55, 'petrol': 56, 'inform': 57, 'carpet': 58, 'show': 59, 'full': 60, 'friend': 61, 'light': 62, 'glove': 63, 'feast': 64, 'garden': 65, 'wall': 66, 'right': 67, 'sister': 68, 'goodbye': 69, 'no': 70, 'friday': 71, 'towel': 72, 'shopping': 73, 'father': 74, 'enemy': 75, 'halal': 76, 'family': 77, 'mother': 78, 'get_well': 79, 'I': 80, 'grandfather': 81, 'bring': 82, 'past': 83, 'wedding': 84, 'see': 85, 'house': 86, 'kin': 87, 'married': 88, 'needle': 89, 'look': 90, 'work': 91, 'lake': 92, 'hungry': 93, 'photograph': 94, 'brother': 95, 'never': 96, 'teapot': 97, 'congratulations': 98, 'doctor': 99}
    for x,y in code.items():
        if n==y:
            return x
def getcode(n):
    code={'Ain': 0, 'Al': 1, 'Alef': 2, 'Beh': 3, 'Dad': 4, 'Dal': 5, 'Feh': 6, 'Ghain': 7, 'Hah': 8, 'Heh': 9, 'Jeem': 10, 'Kaf': 11, 'Khah': 12, 'Laa': 13, 'Lam': 14, 'Meem': 15, 'Noon': 16, 'Qaf': 17, 'Reh': 18, 'Sad': 19, 'Seen': 20, 'Sheen': 21, 'Tah': 22, 'Teh': 23, 'Teh_Marbuta': 24, 'Thal': 25, 'Theh': 26, 'Waw': 27, 'Yeh': 28, 'Zah': 29, 'Zain': 30,'nothing' : 31}
    for x,y in code.items():
        if n==y:
            return x
def getcodeen(n):
    code={'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}
    for x,y in code.items():
        if n==y:
            return x

@app.route('/words', methods=['POST'])
def words():
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    image_value={}
    for key, value in frames[0].items():
        image_value = value
    image_data = base64.b64decode(image_value['Image'])
    img = Image.open(BytesIO(image_data))
    image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
    counter+=1
    img.save(image_path, format="JPEG")
    framelist.append(f'{image_path}')
    
    (input_IMG, output_IMG) = extract_feature(framelist[0])
    input_IMG=input_IMG.reshape(-1,1)
    input_IMG = np.expand_dims(input_IMG, axis=0)
    # Load the TensorFlow Lite model
    interpreter = tf.lite.Interpreter(model_path=model_file2)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_shape = input_details[0]['shape']
    input_data = np.array(input_IMG, dtype=np.float32)
    if not np.all(input_data == 0):
      # Set input tensor
      interpreter.set_tensor(input_details[0]['index'], input_data)

      # Run inference
      interpreter.invoke()

      # Get output tensor
      output_data = interpreter.get_tensor(output_details[0]['index'])
    
      class_index = np.argmax(output_data)
    else : 
        class_index = 31
    label=getcode(class_index)
    return jsonify({'placement':str(label)})

@app.route('/wordsen', methods=['POST'])
def wordsen():
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    image_value={}
    for key, value in frames[0].items():
        image_value = value
    image_data = base64.b64decode(image_value['Image'])
    img = Image.open(BytesIO(image_data))
    image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
    counter+=1
    img.save(image_path, format="JPEG")
    framelist.append(f'{image_path}')
    
    (input_IMG, output_IMG) = extract_feature(framelist[0])
    input_IMG=input_IMG.reshape(-1,1)
    input_IMG = np.expand_dims(input_IMG, axis=0)
    # Load the TensorFlow Lite model
    interpreter = tf.lite.Interpreter(model_path=model_file3)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_shape = input_details[0]['shape']
    input_data = np.array(input_IMG, dtype=np.float32)
    if not np.all(input_data == 0):
      # Set input tensor
      interpreter.set_tensor(input_details[0]['index'], input_data)

      # Run inference
      interpreter.invoke()

      # Get output tensor
      output_data = interpreter.get_tensor(output_details[0]['index'])
    
      class_index = np.argmax(output_data)
    else : 
        class_index = 27
    label=getcodeen(class_index)
    return jsonify({'placement':str(label)})

# Function to Extract Feature from images or Frame
def extract_feature(input_image):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    image = cv2.imread(input_image)

    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.1) as hands:
        while True:
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
            image_height, image_width, _ = image.shape

            if not results.multi_hand_landmarks:


                hand_data=np.zeros(63)

                # Set image to Zero
                annotated_image = 0

                # Return Whole Landmark and Image
                return (hand_data,
                        annotated_image)

            annotated_image = cv2.flip(image.copy(), 1)
            hand_data = []
            # Iterate over all hands detected in the image.
            for hand_landmarks in results.multi_hand_landmarks:

                # Extract the coordinates of all 21 hand landmarks.
                landmark_coordinates = []
                for landmark in mp_hands.HandLandmark:
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].x * image_width)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].y * image_height)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].z)
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Add the coordinates of all 21 hand landmarks for the current hand to the new array.
                hand_data.extend(landmark_coordinates)


            hand_data = np.array(hand_data)
            return  (hand_data, annotated_image)
@app.route('/wordstur', methods=['POST'])
def wordstur():
    np.set_printoptions(precision=3, suppress=True)
    data = request.get_json()
    frames = data['Frames']
    framelist = []
    counter = 0
    # Loop over the list and process each bitmap object
    image_value={}
    for key, value in frames[0].items():
        image_value = value
    image_data = base64.b64decode(image_value['Image'])
    img = Image.open(BytesIO(image_data))
    image_path = f"{app.config['UPLOAD_FOLDER']}{counter}.jpg"
    counter+=1
    img.save(image_path, format="JPEG")
    framelist.append(f'{image_path}')
    path_to_image =  framelist[0]
    (input_IMG, output_IMG) = extract_featureTur(path_to_image)
    input_IMG=input_IMG.reshape(-1,1)
    input_IMG = np.expand_dims(input_IMG, axis=0)
    # Load the TensorFlow Lite model
    interpreter = tf.lite.Interpreter(model_path=modelTur)
    interpreter.allocate_tensors()
    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Prepare input data
    input_shape = input_details[0]['shape']
    input_data = np.array(input_IMG, dtype=np.float32)
    if not np.all(input_data == 0):
        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Get output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        out=np.argmax(output_data)
    else :
        out = 24
    return jsonify({'placement':str(getcodeTurk(out))})
def getcodeTurk(n):
    code={'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'R': 16, 'S': 17, 'T': 18, 'U': 19, 'V': 20, 'Y': 21, 'Z': 22, 'del': 23, 'nothing': 24, 'space': 25}
    for x,y in code.items():
        if n==y:
            return x
# Function to Extract Feature from images or Frame
def extract_featureTur(input_image):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    image = cv2.imread(input_image)

    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.1) as hands:
        while True:
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
            image_height, image_width, _ = image.shape
            # Print handedness (left v.s. right hand).
            # Caution : Uncomment these print command will resulting long log of mediapipe log
            #print(f'Handedness of {input_image}:')
            #print(results.multi_handedness)

            # Draw hand landmarks of each hand.
            # Caution : Uncomment these print command will resulting long log of mediapipe log
            #print(f'Hand landmarks of {input_image}:')
            if not results.multi_hand_landmarks:
                # Here we will set whole landmarks into zero as no handpose detected
                # in a picture wanted to extract.

                # Wrist Hand
                hand_data=np.zeros(10)

                # Set image to Zero
                annotated_image = 0

                # Return Whole Landmark and Image
                return (hand_data,
                        annotated_image)

            annotated_image = cv2.flip(image.copy(), 1)
            hand_data = []
            # Iterate over all hands detected in the image.
            for hand_landmarks in results.multi_hand_landmarks:

                # Extract the coordinates of all 21 hand landmarks.
                landmark_coordinates = []
                for landmark in mp_hands.HandLandmark:
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].x * image_width)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].y * image_height)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].z)
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Add the coordinates of all 21 hand landmarks for the current hand to the new array.
                hand_data.extend(landmark_coordinates)
            #If less than two hands are detected, fill the rest of the array with zeros.
            if len(results.multi_hand_landmarks) < 2:
                num_missing_hands = 2 - len(results.multi_hand_landmarks)
                num_missing_coordinates = num_missing_hands * 21 * 3  # 21 landmarks * 3 coordinates (x, y, z)
                hand_data.extend([0] * num_missing_coordinates)

            hand_data = np.array(hand_data)
            return  (hand_data, annotated_image)
def combine_videos(video_paths, output_path):
    # Open the first video to get video properties
    first_video = cv2.VideoCapture(video_paths[0])
    frame_width = int(first_video.get(3))
    frame_height = int(first_video.get(4))
    fps = first_video.get(5)
    first_video.release()

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can choose the codec based on your needs
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Iterate through each video and append frames to the output video
    for video_path in video_paths:
        video = cv2.VideoCapture(video_path)
        while True:
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)

        video.release()

    # Release the output video
    out.release()

def video_to_base64(video_path):
    try:
        with open(video_path, 'rb') as video_file:
            video_binary = video_file.read()

        video_base64 = base64.b64encode(video_binary).decode('utf-8')

        return video_base64

    except Exception as e:
        print(f'Error encoding video: {str(e)}')
        return None

def searchinVids(param, lang_words):
    path = 'C:/Users/DELL/Downloads/signApp/langs/greek/'+lang_words+'/'+param+'.mp4'
    if os.path.exists(path):
        result = path
#     else:
#         result = {'word':param,'vid':''}

    return result


@app.route('/text_to_sign', methods=['POST'])
def fun4():
    body = request.get_json()['message']['nameValuePairs']
    try:
        words= body['q']
        cat = body['cat']
    except Exception:
        print(body)
    words = words.split(' ')
    vids = []
    for word in words:
        if 'english'in cat:
            vids.append(searchinVids(word.upper(),cat))
        else :
            vids.append(searchinVids(word,cat))
    combine_videos(vids,f'{app.config["UPLOAD_FOLDER"]}vid.mp4')
    result = video_to_base64(f'{app.config["UPLOAD_FOLDER"]}vid.mp4')
    return jsonify({'placement':str(result)})



if __name__ == '__main__':
    app.run(debug=True, use_reloader=False,host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://192.168.43.177:5000/ (Press CTRL+C to quit)
192.168.43.1 - - [26/Dec/2023 02:25:07] "POST /text_to_sign HTTP/1.1" 200 -
192.168.43.1 - - [26/Dec/2023 02:26:14] "POST /text_to_sign HTTP/1.1" 200 -


In [ ]:
# from flask import Flask, request, jsonify
# import base64
# import os
# import cv2
# app = Flask(__name__)
# UPLOAD_FOLDER = 'videos'
# # Create the folder if it does not exist
# os.makedirs(UPLOAD_FOLDER, exist_ok=True)
# # Set the upload folder as an app configuration
# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
# import cv2

# def combine_videos(video_paths, output_path):
#     # Open the first video to get video properties
#     first_video = cv2.VideoCapture(video_paths[0])
#     frame_width = int(first_video.get(3))
#     frame_height = int(first_video.get(4))
#     fps = first_video.get(5)
#     first_video.release()

#     # Define the codec and create VideoWriter object
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can choose the codec based on your needs
#     out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

#     # Iterate through each video and append frames to the output video
#     for video_path in video_paths:
#         video = cv2.VideoCapture(video_path)
#         while True:
#             ret, frame = video.read()
#             if not ret:
#                 break
#             out.write(frame)

#         video.release()

#     # Release the output video
#     out.release()

# def video_to_base64(video_path):
#     try:
#         with open(video_path, 'rb') as video_file:
#             video_binary = video_file.read()

#         video_base64 = base64.b64encode(video_binary).decode('utf-8')

#         return video_base64

#     except Exception as e:
#         print(f'Error encoding video: {str(e)}')
#         return None

# def searchinVids(param, lang_words):
#     path = 'C:/Users/DELL/Downloads/signApp/langs/greek/'+lang_words+'/'+param+'.mp4'
#     result = ''
#     if os.path.exists(path):
#         result = path
# #     else:
# #         result = {'word':param,'vid':''}

#     return result


# @app.route('/text_to_sign', methods=['POST'])
# def fun4():
#     body = request.get_json()['message']['nameValuePairs']
#     try:
#         words= body['q']
#         cat = body['cat']
#     except Exception:
#         print(body)
#     words = words.split(' ')
#     vids = []
#     for word in words:
#         if 'english'in cat:
#             if word is None:
#                 continue
#             else: 
#                 vids.append(searchinVids(word.upper()))
#         else :
#             vids.append(searchinVids(word,cat))
#     combine_videos(vids,f'{app.config["UPLOAD_FOLDER"]}vid.mp4')
#     result = video_to_base64(f'{app.config["UPLOAD_FOLDER"]}vid.mp4')
#     return jsonify({'placement':str(result)})


# if __name__ == '__main__':
#     app.run(debug=(True), use_reloader=False , host='0.0.0.0')

# # app.config['UPLOAD_FOLDER']


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://192.168.43.177:5000/ (Press CTRL+C to quit)
192.168.43.1 - - [26/Dec/2023 01:39:42] "POST /predictTrain HTTP/1.1" 404 -
192.168.43.177 - - [26/Dec/2023 01:40:02] "GET / HTTP/1.1" 404 -
192.168.43.177 - - [26/Dec/2023 01:40:02] "GET /favicon.ico HTTP/1.1" 404 -
192.168.43.1 - - [26/Dec/2023 01:40:37] "POST /predictTrain HTTP/1.1" 404 -


UsageError: Line magic function `%tp` not found.
